# Primeira Fase - Exploração Inicial

*Importando as Bibliotecas*

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

### 1.Limpeza e Organização do Dataset

In [3]:
data = pd.read_csv('archive/male_players.csv',low_memory=False)

#### 1.1.Filtrando os dados relevantes para análises estatísticas

In [9]:
columns_irrelevant = ["fifa_update","value_eur","wage_eur","age","height_cm","weight_kg","player_url",
                      "update_as_of", "short_name", "long_name", "player_positions",'potential',
                      "dob", "club_team_id", "club_name", "league_id", "league_name", "league_level", "club_position", "club_jersey_number",
                      "club_loaned_from", "club_joined_date", 
                      "club_contract_valid_until_year", "nationality_id", "nationality_name", "nation_team_id", "nation_position", "nation_jersey_number",
                      "preferred_foot", "weak_foot", "skill_moves", "international_reputation", "work_rate", "body_type", "real_face", 
                      "release_clause_eur", "player_tags", "player_traits","player_id"]

players_attributes = data.drop(columns=columns_irrelevant)
players_attributes

,player_id,fifa_version,overall,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,24.0,91,97.0,90.0,80.0,92.0,36.0,78.0,78,...,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,24.0,91,89.0,93.0,66.0,80.0,45.0,88.0,47,...,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,24.0,91,72.0,88.0,94.0,87.0,65.0,78.0,95,...,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
3,158023,24.0,90,80.0,87.0,90.0,94.0,33.0,64.0,83,...,63+3,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,19+3
4,165153,24.0,90,79.0,88.0,83.0,87.0,39.0,78.0,75,...,64+3,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180016,220806,15.0,41,66.0,25.0,29.0,30.0,39.0,53.0,25,...,38,38,38,37,40,41,41,41,40,10
180017,225509,15.0,41,64.0,41.0,27.0,35.0,27.0,41.0,22,...,30,30,30,31,32,31,31,31,32,14
180018,217591,15.0,40,58.0,27.0,35.0,31.0,52.0,68.0,49,...,46,46,46,48,51-1,53-3,53-3,53-3,51-1,12
180019,222947,15.0,40,72.0,27.0,27.0,30.0,35.0,61.0,23,...,36,36,36,37,39,40,40,40,39,13


In [10]:
descricao = players_attributes.describe()
descricao

,player_id,fifa_version,overall,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,180021.000000,180021.000000,180021.000000,159997.000000,159997.000000,159997.000000,159997.000000,159997.000000,159997.000000,180021.000000,...,147133.000000,180021.000000,180021.000000,180021.000000,180021.000000,180021.000000,180021.000000,180021.000000,180021.000000,20024.000000
mean,217326.670294,19.621450,65.712711,68.058839,52.268155,56.995731,62.160484,51.232742,64.824322,49.622605,...,57.837807,45.851456,47.759511,45.764783,16.509979,16.260136,16.129902,16.276951,16.615517,38.750300
std,35215.749284,2.838621,7.018104,11.063818,13.920686,10.427677,10.220788,16.580676,9.755666,17.985783,...,12.305995,20.452502,21.309061,20.899683,17.661659,16.846583,16.499513,17.009393,17.971201,10.578237
min,2.000000,15.000000,40.000000,21.000000,14.000000,20.000000,22.000000,14.000000,27.000000,5.000000,...,3.000000,1.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,200759.000000,17.000000,61.000000,62.000000,42.000000,50.000000,56.000000,36.000000,58.000000,38.000000,...,50.000000,26.000000,27.000000,25.000000,8.000000,8.000000,8.000000,8.000000,8.000000,30.000000
50%,222734.000000,20.000000,66.000000,69.000000,54.000000,58.000000,63.000000,56.000000,66.000000,54.000000,...,59.000000,51.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000,40.000000
75%,239858.000000,22.000000,70.000000,76.000000,63.000000,64.000000,69.000000,64.000000,72.000000,63.000000,...,66.000000,63.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,14.000000,46.000000
max,278145.000000,24.000000,94.000000,97.000000,94.000000,94.000000,96.000000,91.000000,92.000000,95.000000,...,96.000000,94.000000,94.000000,95.000000,91.000000,92.000000,95.000000,92.000000,94.000000,68.000000


#### 1.2.Divindo o Dataset por categoria

In [13]:
#Defense

defense_players = data.loc[data["player_positions"].isin(["GK", "CB", "LB", "RB"])]
defense_players = defense_players[["fifa_version", "player_positions", "short_name",
                                   "movement_sprint_speed","defending_marking_awareness","goalkeeping_reflexes" ]]



